# RAG Chatbot

Introduction: Develop a RAG-based chatbot to answer user queries using a set of FAQs. It aims to enhance the customer experience by delivering timely and accurate responses that are consistent with FAQs

Notebook Purpose: Exploratory and experimental work for designing the RAG pipeline.

Task Breakdown:
1. [Notebook] Data Preparation 
    - FAQ preparation
    - Test data preparation
2. [Notebook] Document Ingestion Pipeline
    - FAQ preprocessing
    - FAQ chunking strategy
    - Select  potental embedding models
    - Develop the document ingestion pipeline
3. [Notebook] RAG Pipeline
    - Develop the retrieval pipeline
    - Evaluate and identify the optimal embedding model
    - Develop the response generation pipeline
    - Develop the RAG pipeline
    - Evaluate the performance of the RAG

In [1]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import pandas as pd
pd.set_option('display.max_colwidth', None)

from haystack import Document
from src.module.document_ingestion_pipeline import DocumentIngestionPipeline
from src.module.retrieval_pipeline import RetrievalPipeline
from src.module.response_generation_pipeline import ResponseGenerationPipeline
from src.module.rag_pipeline import RAGPipeline
from src.utils import load_faqs
from src.constants import DATA_FOLDER, FAQ_FOLDER, PROMPT_TEMPLATE
from data.test_data.test_data import POS_QUES, NEG_QUES

DATA_DIR = os.path.join(parent_dir, DATA_FOLDER)

/opt/miniconda3/envs/rag_malay/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## 1.0 Data Preparation

### 1.1 FAQ Preparation

Each FAQ in the provided Google Doc is separated into individual markdown files. Markdown is selected to preserve the hyperlink in one of the FAQs. 

Please refer to `data/faq` folder.

### 1.2 Test Data Preparation

Test data is created for evaluating the performance of the RAG chatbot in answering user queries. The test data must contain positive questions and negative questions
- 3 Positive question: Queries that can be answered using the FAQs. 3 questions are randomly chosen from the provided FAQs and rephrased into new questions with similar user intent.
- 3 Negative question: Queries that cannot be answered using the FAQs

Please refer to `data/test_data` folder

## 2.0 Document Ingestion Pipeline

### 2.1 FAQ preprocessing
Since the FAQ content is already well-structured and clean, no cleaning or preprocessing was done.

### 2.2 Chunking Strategy

Skipped. Since the FAQs have already been separated into individual files in Step 1.1 (i.e. logically chunked at the document level), no additional chunking will be applied to each FAQ for the following reasons:

- Token length of FAQ is well within the limit of the selected embedding models. The longest FAQ remains well below the embedding model’s maximum token limit (e.g. 514 tokens). Therefore, further chunking is not necessary

- Preserve the full semantic meaning of each question-answer pair. Keeping each FAQ as a single unit preserves its complete semantic meaning. Additional chunking could fragment related information and potentially reduce retrieval accuracy.

### 2.3 Embedding Models

The following multilingual embedding models were explored for this project as they support multiple languages (including Bahasa Melayu) and have decent performance on the Hugging Face Embedding Leaderboard:
- BAAI/bge-m3 | Max token: 8194
- BAAI/bge-multilingual-gemma2 | Max token: 8192

### 2.3 Develop Document Ingestion Pipeline

Code can be found in `model/module/document_ingestion_pipeline.py`

A sample pipeline is created below to ensure the code can be run sucessfully

In [2]:
faqs = [Document(content=faq) for faq in load_faqs(os.path.join(DATA_DIR, FAQ_FOLDER))]
indexing_pipeline = DocumentIngestionPipeline(hf_embedding_model="BAAI/bge-m3")
vector_store = indexing_pipeline.create_vector_store(faqs)
print(vector_store.count_documents())

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
Calculating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.35s/it]
100it [00:00, 17232.85it/s]          

7


Remark: Vector store with 7 documents/chunks is created successfully

## 3.0 RAG Pipeline

### 3.1 Develop retrieval pipeline

Code can be found in `model/module/retrieval_pipeline.py`

A sample pipeline is created below to ensure the code can be run sucessfully

In [3]:
retrieval_pipeline = RetrievalPipeline(
    document_store=vector_store, 
    hf_embedding_model="BAAI/bge-m3",
    top_k=3
)

In [4]:
retrieved_contexts = retrieval_pipeline.retrieve_contexts(query='How to subscribe')
df = pd.DataFrame([doc.to_dict() for doc in retrieved_contexts])
df[['content', 'score']]

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.


,content,score
0,"Question: Mengapa ralat muncul di skrin semasa saya menikmati Tonton?\n\nAnswer: Jika anda ingin mengaktifkan semula langganan untuk\nkandungan eksklusif, anda boleh melawati pautan ini\nhttps://www.tonton.com.my/tontonup untuk menaik taraf akaun anda.\nSebaik sahaja anda mengklik pautan, anda akan melihat halaman yang\nmenunjukkan ruangan ""SUBSCRIBE NOW"".\n\nSila klik ke ruangan ""SUBSCRIBE NOW"" di bahagian ATAS sekali untuk\nmeneruskan proses langganan dan pembayaran.\n\nSila pilih kaedah pembayaran pilihan anda dan ikuti arahan seterusnya di\ndalam Langkah 2.\n\nSebaik sahaja anda telah membuat pembayaran langganan, kami\nmencadangkan anda untuk klik butang Pembaharuan\n",0.563989
1,"Question: Bagaimana saya nak membatalkan langganan bulanan TontonUp saya?\nAnswer:\nUntuk makluman, anda boleh membatalkan pembayaran berulang\nautomatik anda pada Profil Tonton anda. Bagi rujukan mudah anda, berikut\nadalah langkah-langkah untuk membatalkan langganan TontonUp anda:\nAnda boleh menggunakan TELEFON BIMBIT atau DESKTOP melalui\nlaman web (komputer).\nTelefon Bimbit\n1) Pergi ke Tetapan (Settings) > klik pada Akaun (Account) > Klik pada\nPembaharuan Kelayakan (Refresh Entitlements) > Kemudian klik pada\nUrus Akaun (Manage account)\n2) Klik pada pelan langganan (Subscription Plan)\n3) Klik pada batalkan langganan berulang (Cancel Recurring Subscription)\nLaman Web (Website)\n1) Log masuk ke Akaun (Account) > Klik pada Tetapan (Settings) > Pilih\nAkaun (Account\n2) Pergi ke pelan langganan (Subscription Plan)\n3) Klik pada batalkan langganan berulang (Cancel Recurring Subscription)\nJika anda menghadapi masalah untuk membatalkan langganan anda, sila\nsertakan resit pembayaran terkini dan butiran berikut untuk kami\nmenyemak dengan lebih lanjut dan melakukan pembatalan manual\ndaripada sistem kami.\nAlamat E-mel Berdaftar:\nNombor Telefon Berdaftar:\nUntuk makluman anda, permohonan pembatalan langganan hendaklah\ndilakukan selewat-lewatnya 5 hari sebelum tarikh tamat langganan anda.",0.548228
2,Question: Kenapa saya tidak boleh menonton selepas membuat bayaran?\n\nAnswer:\nKami menyarankan anda supaya mengikuti langkah-langkah berikut untuk\nmengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit)\naplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account)\n4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement)\n5. Tunggu sehingga tanda ✅\n6. Klik logo Tonton untuk kembali ke laman utama.,0.501669


Remark: Retrieval pipeline is created successfully and 3 contexts are retrieved


### 3.2 Evaluate embedding models

As stated above, the following 2 embedding models are considered for this project: 
- BAAI/bge-m3 | Max token: 8194
- BAAI/bge-multilingual-gemma2 | Max token: 8192

To systematically evaluate the performance of the retrieval module, i.e. whether relevant context is retrieved for a given query and whether it is ranked highly, we can use the following metrics:
1. NDCG (Normalized Discounted Cumulative Gain) to measure ranking effectiveness
2. Context Precision and Recall from the Ragas framework, which leverages an LLM-as-a-judge approach

However, due to time constraints, the optimal embedding model will be selected by manual inspection of the retrieval results for a given query

In [5]:
faqs = [Document(content=faq) for faq in load_faqs(os.path.join(DATA_DIR, FAQ_FOLDER))]

# Retrieval pipeline using BAAI/bge-m3
embedding_model = "BAAI/bge-m3"
m3_indexing_pipeline = DocumentIngestionPipeline(hf_embedding_model=embedding_model)
m3_vector_store = m3_indexing_pipeline.create_vector_store(faqs)
m3_retrieval_pipeline = RetrievalPipeline(
    document_store=m3_vector_store, 
    hf_embedding_model=embedding_model,
    top_k=3
)

# Retrieval pipeline using BAAI/bge-multilingual-gemma2
embedding_model = "BAAI/bge-multilingual-gemma2"
gemma2_indexing_pipeline = DocumentIngestionPipeline(hf_embedding_model=embedding_model)
gemma2_vector_store = gemma2_indexing_pipeline.create_vector_store(faqs)
gemma2_retrieval_pipeline = RetrievalPipeline(
    document_store=gemma2_vector_store, 
    hf_embedding_model=embedding_model,
    top_k=3
)

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.32it/s]
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
Calculating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]
100it [00:00, 15506.32it/s]          
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
Calculating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
Calculating embeddings: 100%|██████████| 1/1

#### 3.2.1 Evaluate "BAAI/bge-m3"

In [6]:
df_output = pd.DataFrame()

for qna_pair in POS_QUES:
    question = qna_pair['Question']

    m3_retrieved_context = m3_retrieval_pipeline.retrieve_contexts(question)
    df = pd.DataFrame([doc.to_dict() for doc in m3_retrieved_context])
    df['question'] = question
    df = df[['question', 'content', 'score']]
    
    df_output = pd.concat([df_output, df], ignore_index=True)

df_output

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.


,question,content,score
0,"Lepas bayar, perlu buat apa untuk mula menonton?",Question: Kenapa saya tidak boleh menonton selepas membuat bayaran?\n\nAnswer:\nKami menyarankan anda supaya mengikuti langkah-langkah berikut untuk\nmengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit)\naplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account)\n4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement)\n5. Tunggu sehingga tanda ✅\n6. Klik logo Tonton untuk kembali ke laman utama.,0.720931
1,"Lepas bayar, perlu buat apa untuk mula menonton?","Question: Mengapa ralat muncul di skrin semasa saya menikmati Tonton?\n\nAnswer: Jika anda ingin mengaktifkan semula langganan untuk\nkandungan eksklusif, anda boleh melawati pautan ini\nhttps://www.tonton.com.my/tontonup untuk menaik taraf akaun anda.\nSebaik sahaja anda mengklik pautan, anda akan melihat halaman yang\nmenunjukkan ruangan ""SUBSCRIBE NOW"".\n\nSila klik ke ruangan ""SUBSCRIBE NOW"" di bahagian ATAS sekali untuk\nmeneruskan proses langganan dan pembayaran.\n\nSila pilih kaedah pembayaran pilihan anda dan ikuti arahan seterusnya di\ndalam Langkah 2.\n\nSebaik sahaja anda telah membuat pembayaran langganan, kami\nmencadangkan anda untuk klik butang Pembaharuan\n",0.626613
2,"Lepas bayar, perlu buat apa untuk mula menonton?","Question: Saya telah melanggan tetapi kenapa masih mempunyai iklan?\n\nAnswer:\nUntuk makluman anda, langganan TontonUp memberikan anda akses\nuntuk menonton filem dan drama eksklusif dan saluran TV premium yang\nhanya boleh didapati di aplikasi Tonton. Untuk menonton tanpa iklan, anda\nperlulah memilih salah satu daripada saluran TV premium dan bahan\ntontonan eksklusif yang bertanda logo TontonUp.\nUntuk cerita tontonan yang bertanda TontonUp, iklan hanya disiarkan pada\nepisod pertama sehingga ke tiga bagi drama yang ditonton sahaja dan\ntidak akan mempunyai iklan pada episod seterus.",0.586341
3,Reset password,"Question: Bagaimana saya nak menukar kata laluan?\nAnswer: Kami mencadangkan anda untuk membuat tetapan semula kata\nlaluan, kemudian log masuk semula.\n\nAnda boleh ke pautan di bawah bagi menetapkan semula kata laluan bagi\nakaun anda - [Laman](https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB)",0.675699
4,Reset password,"Question: Mengapa ralat muncul di skrin semasa saya menikmati Tonton?\n\nAnswer: Jika anda ingin mengaktifkan semula langganan untuk\nkandungan eksklusif, anda boleh melawati pautan ini\nhttps://www.tonton.com.my/tontonup untuk menaik taraf akaun anda.\nSebaik sahaja anda mengklik pautan, anda akan melihat halaman yang\nmenunjukkan ruangan ""SUBSCRIBE NOW"".\n\nSila klik ke ruangan ""SUBSCRIBE NOW"" di bahagian ATAS sekali untuk\nmeneruskan proses langganan dan pembayaran.\n\nSila pilih kaedah pembayaran pilihan anda dan ikuti arahan seterusnya di\ndalam Langkah 2.\n\nSebaik sahaja anda telah membuat pembayaran langganan, kami\nmencadangkan anda untuk klik butang Pembaharuan\n",0.526842
5,Reset password,Question: Kenapa saya tidak boleh menonton selepas membuat bayaran?\n\nAnswer:\nKami menyarankan anda supaya mengikuti langkah-langkah berikut untuk\nmengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit)\naplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account)\n4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement)\n5. Tunggu sehingga tanda ✅\n6. Klik logo Tonton untuk kembali ke laman utama.,0.475314
6,Skrin t

Findings for BAAI/bge-m3:
1. For the question, "Lepas bayar, perlu buat apa untuk mula menonton?", the correct context is retrieved and ranked first
2. For the question, "Reset password", the correct context is retrieved and ranked first. This shows that although the question is in English, BAAI/bge-m3 is able to retrieve the relevant context in Malay
3. For the question, "Skrin tunjuk error code", the correct context is retrieved and ranked first

Takeaway for BAAI/bge-m3: BAAI/bge-m3 is an adequate embedding model for this set of FAQs

#### 3.2.2 BAAI/bge-multilingual-gemma2

In [7]:
df_output = pd.DataFrame()

for qna_pair in POS_QUES:
    question = qna_pair['Question']

    m3_retrieved_context = gemma2_retrieval_pipeline.retrieve_contexts(question)
    df = pd.DataFrame([doc.to_dict() for doc in m3_retrieved_context])
    df['question'] = question
    df = df[['question', 'content', 'score']]
    
    df_output = pd.concat([df_output, df], ignore_index=True)

df_output

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.


,question,content,score
0,"Lepas bayar, perlu buat apa untuk mula menonton?",Question: Kenapa saya tidak boleh menonton selepas membuat bayaran?\n\nAnswer:\nKami menyarankan anda supaya mengikuti langkah-langkah berikut untuk\nmengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit)\naplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account)\n4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement)\n5. Tunggu sehingga tanda ✅\n6. Klik logo Tonton untuk kembali ke laman utama.,0.748645
1,"Lepas bayar, perlu buat apa untuk mula menonton?","Question: Boleh saya melanggan apabila saya di luar negara Malaysia?\n\nAnswer: Untuk makluman, platform kami hanya boleh di tonton di\nMalaysia sahaja pada ketika ini.",0.686689
2,"Lepas bayar, perlu buat apa untuk mula menonton?","Question: Bagaimana saya nak menukar kata laluan?\nAnswer: Kami mencadangkan anda untuk membuat tetapan semula kata\nlaluan, kemudian log masuk semula.\n\nAnda boleh ke pautan di bawah bagi menetapkan semula kata laluan bagi\nakaun anda - [Laman](https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB)",0.657002
3,Reset password,"Question: Bagaimana saya nak menukar kata laluan?\nAnswer: Kami mencadangkan anda untuk membuat tetapan semula kata\nlaluan, kemudian log masuk semula.\n\nAnda boleh ke pautan di bawah bagi menetapkan semula kata laluan bagi\nakaun anda - [Laman](https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB)",0.687109
4,Reset password,Question: Kenapa saya tidak boleh menonton selepas membuat bayaran?\n\nAnswer:\nKami menyarankan anda supaya mengikuti langkah-langkah berikut untuk\nmengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit)\naplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account)\n4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement)\n5. Tunggu sehingga tanda ✅\n6. Klik logo Tonton untuk kembali ke laman utama.,0.510969
5,Reset password,"Question: Boleh saya melanggan apabila saya di luar negara Malaysia?\n\nAnswer: Untuk makluman, platform kami hanya boleh di tonton di\nMalaysia sahaja pada ketika ini.",0.507417
6,Skrin tunjuk error code,"Question: Bagaimana saya nak menukar kata laluan?\nAnswer: Kami mencadangkan anda untuk membuat tetapan semula kata\nlaluan, kemudian log masuk semula.\n\nAnda boleh ke pautan di bawah bagi menetapkan semula kata laluan bagi\nakaun anda - [Laman](https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB)",0.635778
7,Skrin tunjuk error code,Question: Kenapa saya tidak boleh menonton selepas membuat bayaran?\n\nAnswer:\nKami menyarankan anda supaya mengikuti langkah-langkah berikut untuk\nmengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit)\naplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account)\n4. Klik b

Findings for BAAI/bge-multilingual-gemma2:
1. For the question, "Lepas bayar, perlu buat apa untuk mula menonton?", the correct context is retrieved and ranked first
2. For the question, "Reset password", the correct context is retrieved and ranked first. This shows that although the question is in English, BAAI/bge-multilingual-gemma2 is able to retrieve the relevant context in Malay
3. For the question, "Skrin tunjuk error code", the correct context is retrieved but it is only ranked third

Takeaway for BAAI/bge-multilingual-gemma2: 
1. BAAI/bge-multilingual-gemma2 perform slightly worse than BAAI/bge-m3 because for the third question, the most relevant context is only ranked third by bge-multilingual-gemma2


Overall Takeaway:
1. BAAI/bge-m3 is found to be a better embedding model for this given FAQ and thus, will be used for this project
2. Given that BAAI/bge-m3 is able to retrieve the correct contexts and rank them first, reranker will not be considered for this project for now

### 3.3 Develop response generation pipeline

Code can be found in `model/module/response_generation_pipeline.py`

A sample pipeline is created below to ensure the code can be run sucessfully


In [8]:
response_generation_pipeline = ResponseGenerationPipeline(prompt_template=PROMPT_TEMPLATE)
question = 'Lepas bayar, perlu buat apa untuk mula menonton?'
m3_retrieved_context = m3_retrieval_pipeline.retrieve_contexts(question)
answer = response_generation_pipeline.generate_response(question, m3_retrieved_context)

print(f"Question:\n{question}\n")
print(f"Answer:\n{answer}")

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.


Question:
Lepas bayar, perlu buat apa untuk mula menonton?

Answer:
Kami menyarankan anda supaya mengikuti langkah-langkah berikut untuk mengaktifkan langganan anda selepas bayaran dilakukan:
1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit) aplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.
2. Pilih Tetapan Aplikasi (App Settings) daripada menu.
3. Klik butang Akaun (Account)
4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement)
5. Tunggu sehingga tanda ✅
6. Klik logo Tonton untuk kembali ke laman utama.


In [9]:
question = 'Contact kustomer servis'
m3_retrieved_context = m3_retrieval_pipeline.retrieve_contexts(question)
answer = response_generation_pipeline.generate_response(question, m3_retrieved_context)

print(f"Question:\n{question}\n")
print(f"Answer:\n{answer}")

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.


Question:
Contact kustomer servis

Answer:
Tidak ada maklumat


Remark: Response is generated successfully

### 3.4 Develop RAG pipeline

Combine retrieval pipeline and response generation pipeline into a single RAG pipeline

Code can be found in `model/module/rag_pipeline.py`

A sample pipeline is created below to ensure the code can be run sucessfully



In [10]:
rag_pipeline = RAGPipeline(
    m3_retrieval_pipeline,
    response_generation_pipeline
)
rag_pipeline.answer(question = 'Contact kustomer servis')


`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.


{'status': 'success',
 'error_message': None,
 'question': 'Contact kustomer servis',
 'answer': 'Untuk maklumat lanjut, sila hubungi kami secara terus melalui saluran berikut:\n\n*   **Telefon Bimbit:**\n    1.  Pergi ke Tetapan (Settings) > klik pada Akaun (Account) > Klik pada Pembaharuan Kelayakan (Refresh Entitlements) > Kemudian klik pada Urus Akaun (Manage account)\n    2.  Klik pada pelan langganan (Subscription Plan)\n    3.  Klik pada batalkan langganan berulang (Cancel Recurring Subscription)\n*   **Laman Web (Website):**\n    1.  Log masuk ke Akaun (Account) > Klik pada Tetapan (Settings) > Pilih Akaun (Account)\n    2.  Pergi ke pelan langganan (Subscription Plan)\n    3.  Klik pada batalkan langganan berulang (Cancel Recurring Subscription)\n\nJika anda menghadapi masalah untuk membatalkan langganan anda, sila sertakan resit pembayaran terkini dan butiran berikut untuk kami menyemak dengan lebih lanjut dan melakukan pembatalan manual daripada sistem kami:\n*   Alamat E-me

Remark: Response is generated successfully

### Evaluate RAG performance

A RAG consists of 3 important elements, which are the query, the context and the answer. Systematic evaluation involves analyzing the relationships between these pairs:
1. Faithfulness / Hallucination (Answer <> Context): Evaluate whether the answer is generated solely based on the retrieved context
2. Answer Relevance (Answer ↔ Query): Evaluate how well the answer addresses the question
3. Context Precision (Context ↔ Query): Evaluate whether the retrieved context actually contain the information necessary to answer the question

The above can be achieved through Ragas framework, which uses LLM-as-a-judge for evaluation

Howver, due to time constraints, the above is not implemented. Instead, the answer will be inspected manually to evaluate the performance of the RAG

In [11]:
output = {
    "question": [],
    "answer": []
}

for qna_pair in POS_QUES + NEG_QUES:
    question = qna_pair['Question']
    rag_output = rag_pipeline.answer(question)

    output["question"].append(question)
    output["answer"].append(rag_output['answer'])
    
df_output = pd.DataFrame(output)

`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is not supported for Serverless Inference API. It will be ignored.
`truncate` parameter is not supported for Serverless Inference API. It will be ignored.
`normalize` parameter is no

In [12]:
df_output

,question,answer
0,"Lepas bayar, perlu buat apa untuk mula menonton?",Kami menyarankan anda supaya mengikuti langkah-langkah berikut untuk mengaktifkan langganan anda selepas bayaran dilakukan:\n1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit) aplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda.\n2. Pilih Tetapan Aplikasi (App Settings) daripada menu.\n3. Klik butang Akaun (Account).\n4. Klik butang Pembaharuan Kelayakan (Refresh Entitlement).\n5. Tunggu sehingga tanda ✅.\n6. Klik logo Tonton untuk kembali ke laman utama.
1,Reset password,Sila ikuti pautan di bawah untuk menetapkan semula kata laluan anda: [Laman](https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB)
2,Skrin tunjuk error code,"Jika ralat muncul di skrin semasa anda menikmati Tonton, anda boleh melawati pautan ini https://www.tonton.com.my/tontonup untuk menaik taraf akaun anda. Sila klik ke ruangan ""SUBSCRIBE NOW"" di bahagian ATAS sekali untuk meneruskan proses langganan dan pembayaran. Sila pilih kaedah pembayaran pilihan anda dan ikuti arahan seterusnya. Sebaik sahaja anda telah membuat pembayaran langganan, kami mencadangkan anda untuk klik butang Pembaharuan."
3,Macam mana nak mohon kerja kosong dekat Tonton,Tidak ada maklumat
4,Berapa caj bulanan,Tidak ada maklumat
5,Contact customer service,Tidak ada maklumat


Findings: Correct response is generated for the test question

## Future Improvements

### Short-Term
1. Add citation link for every answer to increase user trust in the answer
2. Add LLM guardrails to prevent the chatbot from answering malicious question or providing biased answer
3. Develop a systematic evaluation of the RAG
4. Built an automated document ingestion pipeline that ingest documents automatically whenever it detects FAQ updates

### Long-Term
1. Apply semantic cache to answer similar questions that have been answered before in order to reduce cost and response time
2. Introduce conversational memory
3. Agentic RAG, i.e. if the question is a legitimate question and cannot be answered using the FAQs, search information online to answer the question
4. Personalised RAG, i.e. provide a personalised answer to users based on their product usage and personal data